In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
# packacges that needs to be installed across all nodes:
from textblob import TextBlob 
import nltk
from nltk.corpus import stopwords

from collections import Counter
import string
import nltk

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.197:7077") \
        .appName("Hedaya_uc")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","60s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/Reddit.comments") \
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/Reddit.comments") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/home/ubuntu/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-03940523-7c8b-4bd9-b7cc-29acdbe26558;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 935ms :: artifacts dl 17ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts

In [2]:
#starts timing
start_time = time.time()
#path to the file
path = 'hdfs://130.238.28.151:9000/user/hadoop/RC_2010-01'
#reading the json file as a dataframe
df = spark_session.read\
    .option("header", "true")\
    .json(path)\
    .cache()

In [3]:
#df.printSchema()

In [4]:
#df.show()

In [8]:
avg_subreddit_score = df.groupBy("subreddit").agg({'score': 'avg'})

avg_subreddit_score_sorted = avg_subreddit_score.orderBy('avg(score)',ascending = False)


In [10]:
print("The top 20 kindest subreddits based on average comment score are:")
avg_subreddit_score_sorted.show()

The top 20 kindest subreddits based on average comment score are:


22/03/18 17:04:00 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.2.197: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


+--------------+------------------+
|     subreddit|        avg(score)|
+--------------+------------------+
|    bestof2009|41.943885653785074|
|        tattoo|              20.0|
|          digg|              13.0|
|doesanyoneelse| 9.666666666666666|
| announcements| 8.293814432989691|
|         latin|7.7272727272727275|
|      Nautical|              7.25|
|         Forts|               7.0|
|          blog| 6.954072398190045|
|     Lovecraft| 6.333333333333333|
|         funny| 6.329083665338645|
|          xkcd|             6.158|
|         Bacon| 6.102348993288591|
|     malkovich|               6.0|
|    homeopathy|               6.0|
|         Eesti|               6.0|
| worldbuilding| 5.888888888888889|
|        bestof| 5.669226225634968|
|  ChatRoulette|               5.5|
|           WTF| 5.459002403427497|
+--------------+------------------+
only showing top 20 rows



In [7]:
print(f"Execution time: {time.time() - start_time}")

Execution time: 101.16121912002563
